In [92]:
import warnings
warnings.filterwarnings("ignore")

import sklearn
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler
from sklearn import linear_model

import math
import random
import keras
import numpy as np
import pandas as pd
from statsmodels.tsa.arima_model import ARIMA
from datetime import datetime, date
import plotly.offline as py
import plotly.graph_objs as go
py.init_notebook_mode(connected=True)

# Utility functions

In [61]:
def flatten(list):
    return [item for sublist in list for item in sublist]

In [62]:
def RMSE(prediction, actual):
    return math.sqrt(sklearn.metrics.mean_squared_error(prediction, actual))

In [63]:
COLOR_PREDICTION = 'rgb(244, 146, 65)'
COLOR_ACTUAL = 'rgb(66, 244, 155)'

In [64]:
def plot_prediction(prediction, actual, title=''):
    plot_data = []

    plot_data.append(go.Scatter(
        x = np.arange(0, len(prediction), 1),
        y = prediction,
        mode = 'lines+markers',
        name = 'Predicted labels',
        line = dict(color=COLOR_PREDICTION, width=3)
    ))

    plot_data.append(go.Scatter(
        x = np.arange(0, len(actual), 1),
        y = actual,
        mode = 'lines+markers',
        name = 'True labels',
        line = dict(color=COLOR_ACTUAL, width=3)
    ))

    layout = dict(
        title  = title,
        xaxis  = dict(title = 'Day number', titlefont=dict(size=26), tickfont=dict(size=18)),
        yaxis  = dict(title = 'Price, USD', titlefont=dict(size=26), tickfont=dict(size=18)),
        legend = dict(x=-.1, y=1.2, font=dict(size=22), bgcolor='#E2E2E2', bordercolor='#FFFFFF', borderwidth=2))

    fig = dict(data=plot_data, layout=layout)
    py.iplot(fig)

In [65]:
def plot_training_error(history):
    plot_data = []

    plot_data.append(go.Scatter(
        x = np.arange(0, len(history.history['loss']), 1),
        y = history.history['loss'],
        mode = 'lines',
        name = 'Train loss',
        line = dict(color=COLOR_ACTUAL, width=3, dash='dash')
    ))

    plot_data.append(go.Scatter(
        x = np.arange(0, len(history.history['val_loss']), 1),
        y = history.history['val_loss'],
        mode = 'lines',
        name = 'Test loss',
        line = dict(color=COLOR_PREDICTION, width=3)
    ))

    layout = dict(
        xaxis  = dict(title = 'Epoch number', titlefont=dict(size=26), tickfont=dict(size=18)),
        yaxis  = dict(title = 'Loss', titlefont=dict(size=26), tickfont=dict(size=18)),
        legend = dict(x=-.1, y=1.2, font=dict(size=22), bgcolor='#E2E2E2', bordercolor='#FFFFFF', borderwidth=2))

    fig = dict(data=plot_data, layout=layout)
    py.iplot(fig, filename='training_process')

# Data preprocessing

In [66]:
data = pd.read_csv('./Data/bitstampUSD_1-min_data_2012-01-01_to_2018-11-11.csv')
data = data.dropna()

data['date'] = pd.to_datetime(data['Timestamp'], unit='s').dt.date
Daily_Price = data.groupby('date').agg({'Weighted_Price':'mean', 'Volume_(Currency)':'sum'})

print(Daily_Price.head())
print(Daily_Price.tail())

            Weighted_Price  Volume_(Currency)
date                                         
2011-12-31        4.471603         425.320338
2012-01-01        4.806667         105.779160
2012-01-02        5.000000          95.240000
2012-01-03        5.252500         464.805210
2012-01-04        5.208159         568.076197
            Weighted_Price  Volume_(Currency)
date                                         
2018-11-06     6413.277403       2.741794e+07
2018-11-07     6506.850121       3.124798e+07
2018-11-08     6448.400227       2.557049e+07
2018-11-09     6365.113235       2.047625e+07
2018-11-10     6354.594869       9.677159e+06


In [67]:
train_start = date(2016, 1, 1)
train_end = test_start = date(2017, 8, 21)
test_end = date(2017, 10, 20)

df_train = Daily_Price[(Daily_Price.index >= train_start) & (Daily_Price.index <= train_end)]
df_test = Daily_Price[(Daily_Price.index >= test_start) & (Daily_Price.index <= test_end)]
print(df_train.shape, df_test.shape)

df_train_price_only = df_train.iloc[:,0]
df_test_price_only = df_test.iloc[:,0]

(599, 2) (61, 2)


In [68]:
plot_data = []

plot_data.append(go.Scatter(
    x = df_train.index,
    y = df_train_price_only,
    mode = 'lines+markers',
    name = 'Training set',
    line = dict(color=COLOR_ACTUAL, width=2, dash='dash')
))

plot_data.append(go.Scatter(
    x = df_test.index,
    y = df_test_price_only,
    mode = 'lines+markers',
    name = 'Test set',
    line = dict(color=COLOR_PREDICTION, width=2)
))

layout = dict(
    xaxis  = dict(tickfont=dict(size=18)),
    yaxis  = dict(title = 'Price, USD', titlefont=dict(size=26), tickfont=dict(size=18)),
    legend = dict(x=-.1, y=1.1, font=dict(size=22), bgcolor='#E2E2E2', bordercolor='#FFFFFF', borderwidth=2))

fig = dict(data=plot_data, layout=layout)
py.iplot(fig, filename='training_process')

# 1. Single-step prediction

In [69]:
# based on: https://activewizards.com/blog/bitcoin-price-forecasting-with-deep-learning-algorithms
def create_lookback(dataset, look_back=1):
    X, Y = [], []
    for i in range(len(dataset) - look_back):
        X.append(dataset[i:(i + look_back), :])
        Y.append(dataset[i + look_back, 0])
    return np.array(X), np.array(Y)

## 1.1 Naive

In [70]:
pred_naive = df_test_price_only.shift(1).values[1:]
y_naive = df_test_price_only[1:]

In [71]:
plot_prediction(pred_naive, y_naive)

In [72]:
RSME_naive = RMSE(pred_naive, y_naive)
print('Naive RMSE: %.3f' % RSME_naive)

Naive RMSE: 161.024


## 1.2 ARIMA

In [73]:
#"AR", "I" or "MA" ARIMA (1,0,0) is AR(1), ARIMA(0,1,0) is I(1), and ARIMA(0,0,1) is MA(1)
history = list(df_train_price_only.values)
pred_arima = []
for t in range(len(df_test.values)):
    pred = ARIMA(history, order=(1,0,0)).fit(disp=0).forecast()[0]
    pred_arima.append(pred)
    obs = df_test_price_only.values[t]
    history.append(obs)

In [74]:
y_arima = df_test.values[:,0]
RMSE_ARIMA =  RMSE(pred_arima, y_arima)
print('ARIMA RMSE: %.3f' % RMSE_ARIMA)

ARIMA RMSE: 159.945


In [75]:
plot_prediction(flatten(pred_arima), y_arima)

## 1.3 LSTM

### 1.3.1 Prepare dataset

In [76]:
training_set = df_train.values
training_set = np.reshape(training_set, (len(training_set), training_set.ndim))
test_set = df_test.values
test_set = np.reshape(test_set, (len(test_set), test_set.ndim))

# scale training set
price_scaler = MinMaxScaler()
volume_scaler = MinMaxScaler()

price_train = np.reshape(training_set[:,0], (len(training_set), 1))
price_train = price_scaler.fit_transform(price_train)
volume_train = np.reshape(training_set[:,1], (len(training_set), 1))
volume_train = volume_scaler.fit_transform(volume_train)

# training_set = np.concatenate((price_train, volume_train), axis=1)
training_set = price_train

# scale test set
price_test = np.reshape(test_set[:,0], (len(test_set), 1))
price_test = price_scaler.transform(price_test)
volume_test = np.reshape(test_set[:,1], (len(test_set), 1))
volume_test = volume_scaler.transform(volume_test)

# test_set = np.concatenate((price_test, volume_test), axis=1)
test_set = price_test

# create windows
look_back = 1
X_train, Y_train = create_lookback(training_set, look_back)
X_test, Y_test = create_lookback(test_set, look_back)

X_train = np.reshape(X_train, (len(X_train), 1, X_train.shape[1]))
X_test = np.reshape(X_test, (len(X_test), 1, X_test.shape[1]))

### 1.3.2 Define & train model

In [77]:
model = keras.models.Sequential()
model.add(keras.layers.LSTM(256, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(keras.layers.LSTM(256))
model.add(keras.layers.Dense(1))

opt = keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=1e-3, amsgrad=False)

model.compile(loss='mean_squared_error', optimizer=opt)
history = model.fit(X_train, Y_train, epochs=300, batch_size=16, shuffle=False,
                    validation_data=(X_test, Y_test),
                    callbacks = [keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=5e-5, patience=20, verbose=1)])

Train on 598 samples, validate on 60 samples
Epoch 1/300
598/598 [==============================] - 3s 5ms/step - loss: 0.0081 - val_loss: 0.1171
Epoch 2/300
598/598 [==============================] - 1s 2ms/step - loss: 0.0652 - val_loss: 0.2888
Epoch 3/300
598/598 [==============================] - 1s 2ms/step - loss: 0.0124 - val_loss: 0.0407
Epoch 4/300
598/598 [==============================] - 1s 2ms/step - loss: 0.0030 - val_loss: 0.0031
Epoch 5/300
598/598 [==============================] - 1s 2ms/step - loss: 2.0119e-04 - val_loss: 0.0029
Epoch 6/300
598/598 [==============================] - 1s 2ms/step - loss: 2.1433e-04 - val_loss: 0.0018
Epoch 7/300
598/598 [==============================] - 1s 2ms/step - loss: 3.2497e-04 - val_loss: 0.0024
Epoch 8/300
598/598 [==============================] - 1s 2ms/step - loss: 2.0190e-04 - val_loss: 0.0021
Epoch 9/300
598/598 [==============================] - 1s 2ms/step - loss: 2.1648e-04 - val_loss: 0.0021
Epoch 10/300
598/598 [====

### 1.3.3 Evaluate

In [78]:
plot_training_error(history)

In [79]:
pred_lstm = model.predict(X_test)
pred_lstm = price_scaler.inverse_transform(pred_lstm.reshape(-1, 1))
pred_lstm = pred_lstm[:,0]

y_lstm = price_scaler.inverse_transform(Y_test.reshape(-1, 1))
y_lstm = np.array(y_lstm[:,0])

print(pred_lstm.shape, y_lstm.shape)

(60,) (60,)


In [80]:
plot_prediction(pred_lstm, y_lstm)

In [81]:
RMSE_LSTM = RMSE(pred_lstm, y_lstm)
print('LSTM RMSE: %.3f' % RMSE_LSTM)

LSTM RMSE: 159.759


In [82]:
# pre-calculated RMSE values for different look-back sizes
rmse_per_day = [159.816, 226.877, 228.045, 261.151, 251.265, 175.626, 209.257, 206.062, 221.582, 227.840]

plot_data = []

plot_data.append(go.Scatter(
    x = np.arange(0, len(rmse_per_day), 1),
    y = rmse_per_day,
    mode = 'lines+markers',
    line = dict(color=COLOR_PREDICTION, width=3)
))

layout = dict(
    xaxis = dict(title = 'Number of days (look-back)', titlefont=dict(size=26), tickfont=dict(size=18)),
    yaxis = dict(range=[150, 262], title = 'RMSE', titlefont=dict(size=26), tickfont=dict(size=18)),
    legend=dict(x=-.1, y=1.2, font=dict(size=22), bgcolor='#E2E2E2', bordercolor='#FFFFFF', borderwidth=2))

fig = dict(data=plot_data, layout=layout)
py.iplot(fig, filename='training_process')

# 2. Multi-step prediction

In [83]:
WINDOWS_TO_PLOT = [20, 1, 24]

In [84]:
def create_lookback(dataset, look_back=1, look_ahead=1):
    X, Y = [], []
    for i in range(look_back, len(dataset) - look_ahead + 1):
        X.append(dataset[(i - look_back):i])
        Y.append(dataset[i:(i + look_ahead)])
    return np.array(X), np.array(Y)

In [85]:
x, y = create_lookback([1, 2, 3, 4, 5, 6], look_back=3, look_ahead=2)
print(list(zip(x, y)))

[(array([1, 2, 3]), array([4, 5])), (array([2, 3, 4]), array([5, 6]))]


In [86]:
def plot_windows(X_test, Y_test, predicted, windows_to_plot, title=''):
    plot_data = []
    min_y = float('inf')
    max_y = float('-inf')

    for i in range(len(windows_to_plot)):
        w = windows_to_plot[i]
        X = flatten(X_test[w])
        Y = Y_test[w]
        pred = predicted[w]

        look_back = len(X)
        look_ahead = len(Y)
        full_window = look_back + look_ahead

        # first half of window: look back
        plot_data.append(go.Scatter(
            x = np.arange(i * full_window, i * full_window + look_back, 1),
            y = X,
            mode = 'lines+markers',
            name = 'Look-back window',
            showlegend=(i==0),
            line = dict(color=('rgb(0, 0, 0)'), width=3)
        ))

        # second half of window: true labels
        plot_data.append(go.Scatter(
            x = np.arange(i * full_window + look_back - 1, i * full_window + full_window, 1),
            y = np.insert(Y, 0, X[-1], axis=0),
            mode = 'lines+markers',
            name = 'True labels',
            showlegend=(i==0),
            line = dict(color=('rgb(66, 244, 155)'), width=3)
        ))

        # second half of window: predicted labels 
        plot_data.append(go.Scatter(
            x = np.arange(i * full_window + look_back - 1, i * full_window + full_window, 1),
            y = np.insert(pred, 0, X[-1], axis=0),
            mode = 'lines+markers',
            name = 'LSTM labels',
            showlegend=(i==0),
            line = dict(color=('#C4270A'), width=4)
        ))

        local_min = min(min(X), min(Y), min(pred))
        local_max = max(max(X), max(Y), max(pred))

        if (local_min < min_y): min_y = local_min
        if (local_max > max_y): max_y = local_max

    # add vertical lines
    shapes = list()
    for i in range(0, len(windows_to_plot), 1):
        shapes.append({
            'type': 'line',
            'x0': i*28,
            'y0': min_y,
            'x1': i*28,
            'y1': max_y,
            'line': {
                'color': 'rgb(55, 128, 191)',
                'width': 1,
        }})


    layout = dict(shapes=shapes,
                  xaxis = dict(showticklabels=False),
                  yaxis = dict(title = 'Price, USD', titlefont=dict(size=26), tickfont=dict(size=18)),
                  legend=dict(x=-.1, y=1.2, font=dict(size=22), bgcolor='#E2E2E2', bordercolor='#FFFFFF', borderwidth=2),
                  title=title, titlefont=dict(size=30))

    fig = dict(data=plot_data, layout=layout)
    py.iplot(fig, filename='results_demonstrating0')

## 2.1 Naive

In [87]:
def plot_all_windows(X_test, Y_test, pred_linear, pred_svm, pred_arima, windows_to_plot, title=''):
    plot_data = []
    min_y = float('inf')
    max_y = float('-inf')

    for i in range(len(windows_to_plot)):
        w = windows_to_plot[i]
        X = flatten(X_test[w])
        Y = Y_test[w]
        p_linear = pred_linear[w]
        p_svm = pred_svm[w]
        p_arima = pred_arima[w]

        look_back = len(X)
        look_ahead = len(Y)
        full_window = look_back + look_ahead

        # first half of window: look back
        plot_data.append(go.Scatter(
            x = np.arange(i * full_window, i * full_window + look_back, 1),
            y = X,
            mode = 'lines+markers',
            name = 'Look-back window',
            showlegend=(i==0),
            line = dict(color=('rgb(0, 0, 0)'), width=3)
        ))

        # second half of window: true labels
        plot_data.append(go.Scatter(
            x = np.arange(i * full_window + look_back - 1, i * full_window + full_window, 1),
            y = np.insert(Y, 0, X[-1], axis=0),
            mode = 'lines+markers',
            name = 'True labels',
            showlegend=(i==0),
            line = dict(color=('rgb(66, 244, 155)'), width=3)
        ))

        # second half of window: predicted labels (linear regression)
        plot_data.append(go.Scatter(
            x = np.arange(i * full_window + look_back - 1, i * full_window + full_window, 1),
            y = np.insert(p_linear, 0, X[-1], axis=0),
            mode = 'lines+markers',
            name = 'Linear regression',
            showlegend=(i==0),
            line = dict(color=('#C4270A'), width=4)
        ))

        # second half of window: predicted labels (SVM regression)
        plot_data.append(go.Scatter(
            x = np.arange(i * full_window + look_back - 1, i * full_window + full_window, 1),
            y = np.insert(p_svm, 0, X[-1], axis=0),
            mode = 'lines+markers',
            name = 'SVM regression',
            showlegend=(i==0),
            line = dict(color=('#FEE90F'), width=4)
        ))

        # second half of window: predicted labels (ARIMA)
        plot_data.append(go.Scatter(
            x = np.arange(i * full_window + look_back - 1, i * full_window + full_window, 1),
            y = np.insert(p_arima, 0, X[-1], axis=0),
            mode = 'lines+markers',
            name = 'ARIMA',
            showlegend=(i==0),
            line = dict(color=('#D16C10'), width=4)
        ))

        local_min = min(min(X), min(Y), min(p_linear), min(p_svm), min(p_arima))
        local_max = max(max(X), max(Y), max(p_linear), max(p_svm), max(p_arima))

        if (local_min < min_y): min_y = local_min
        if (local_max > max_y): max_y = local_max

    # add vertical lines
    shapes = list()
    for i in range(0, len(windows_to_plot), 1):
        shapes.append({
            'type': 'line',
            'x0': i*28,
            'y0': min_y,
            'x1': i*28,
            'y1': max_y,
            'line': {
                'color': 'rgb(55, 128, 191)',
                'width': 1,
        }})


    layout = dict(
        shapes=shapes,
        xaxis = dict(showticklabels=False),
        yaxis = dict(title = 'Price, USD', titlefont=dict(size=26), tickfont=dict(size=18)),
        legend=dict(x=-.1, y=1.2, font=dict(size=22), bgcolor='#E2E2E2', bordercolor='#FFFFFF', borderwidth=2))

    fig = dict(data=plot_data, layout=layout)
    py.iplot(fig, filename='results_demonstrating0')

In [88]:
def predict_naive(window, pred_size, model):
    train_x = [[x] for x in range(len(window))]
    test_x = [[x] for x in range(train_x[-1][0] + 1, train_x[-1][0] + 1 + pred_size)]
    model.fit(train_x, window)
    prediction = model.predict(test_x)
    return prediction

In [89]:
look_back = 21
look_ahead = 7

In [90]:
test_set = df_test.values[:, 0]
test_set = np.reshape(test_set, (len(test_set), test_set.ndim))

X_test_naive, Y_test_naive = create_lookback(test_set, look_back, look_ahead)
Y_test_naive = Y_test_naive[:,:,0]

### 2.1.1 Linear regression

In [93]:
model = sklearn.linear_model.LinearRegression()
pred_naive_linear = list(map(lambda window: predict_naive(flatten(window), look_ahead, model), X_test_naive))

In [94]:
RMSE_NAIVE_LINEAR = RMSE(flatten(pred_naive_linear), flatten(Y_test_naive))
print('Naive RMSE: %.3f' % RMSE_NAIVE_LINEAR)

Naive RMSE: 543.974


In [95]:
plot_windows(X_test_naive, Y_test_naive, pred_naive_linear, WINDOWS_TO_PLOT)

### 2.1.2 SVM/SVR

In [96]:
model = sklearn.svm.SVR(kernel='rbf', C=1e3, gamma=0.1)
pred_naive_svm = list(map(lambda window: predict_naive(flatten(window), look_ahead, model), X_test_naive))

In [97]:
RMSE_NAIVE_SVM = RMSE(flatten(pred_naive_svm), flatten(Y_test_naive))
print('Naive RMSE: %.3f' % RMSE_NAIVE_SVM)

Naive RMSE: 624.713


In [98]:
plot_windows(X_test_naive, Y_test_naive, pred_naive_svm, WINDOWS_TO_PLOT)

### 2.1.3 ARIMA

In [99]:
def predict_naive_ARIMA(window, pred_size):
    train_x = [[x] for x in range(len(window))]
    #"AR", "I" or "MA" ARIMA (1,0,0) is AR(1), ARIMA(0,1,0) is I(1), and ARIMA(0,0,1) is MA(1)
    model_fit = ARIMA(window, order=(1,0,0)).fit(disp=0)
    pred = model_fit.forecast(steps=pred_size)[0]
    return pred

In [100]:
pred_naive_arima = list(map(lambda window: predict_naive_ARIMA(flatten(window), look_ahead), X_test_naive))

In [101]:
RMSE_NAIVE_ARIMA = RMSE(flatten(pred_naive_arima), flatten(Y_test_naive))
print('Naive RMSE: %.3f' % RMSE_NAIVE_ARIMA)

Naive RMSE: 463.771


In [102]:
plot_windows(X_test_naive, Y_test_naive, pred_naive_arima, WINDOWS_TO_PLOT)

### 2.1.4 Naive comparison

In [103]:
plot_all_windows(X_test_naive, Y_test_naive, pred_naive_linear, pred_naive_svm, pred_naive_arima, WINDOWS_TO_PLOT)

## 2.2 LSTM

### 2.2.1 Prepare dataset

In [104]:
# keep price only
training_set = df_train.values[:, 0]
training_set = np.reshape(training_set, (len(training_set), training_set.ndim))

# keep price only
test_set = df_test.values[:, 0]
test_set = np.reshape(test_set, (len(test_set), test_set.ndim))

#scale datasets
scaler = MinMaxScaler()
training_set = scaler.fit_transform(training_set)
test_set = scaler.transform(test_set)

# create windows
look_back = 21
look_ahead = 7

X_train, Y_train = create_lookback(training_set, look_back, look_ahead)
X_test, Y_test = create_lookback(test_set, look_back, look_ahead)

X_train = np.reshape(X_train, (len(X_train), 1, X_train.shape[1]))
Y_train = Y_train[:,:,0]

X_test = np.reshape(X_test, (len(X_test), 1, X_test.shape[1]))
Y_test = Y_test[:,:,0]

### 2.2.2 Define & train model

In [105]:
model = keras.models.Sequential()
model.add(keras.layers.LSTM(256, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(keras.layers.LSTM(256))
model.add(keras.layers.Dense(look_ahead))

opt = keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)

model.compile(loss='mean_squared_error', optimizer=opt)
history = model.fit(X_train, Y_train, epochs=300, batch_size=16, shuffle=False,
                    validation_data=(X_test, Y_test),
                    callbacks = [keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=5e-5, patience=20, verbose=1)])

Train on 572 samples, validate on 34 samples
Epoch 1/300
572/572 [==============================] - 4s 7ms/step - loss: 0.0031 - val_loss: 0.0328
Epoch 2/300
572/572 [==============================] - 1s 2ms/step - loss: 0.0123 - val_loss: 0.0547
Epoch 3/300
572/572 [==============================] - 1s 2ms/step - loss: 0.0055 - val_loss: 0.0337
Epoch 4/300
572/572 [==============================] - 1s 2ms/step - loss: 0.0048 - val_loss: 0.0307
Epoch 5/300
572/572 [==============================] - 1s 2ms/step - loss: 0.0026 - val_loss: 0.0317
Epoch 6/300
572/572 [==============================] - 1s 2ms/step - loss: 0.0039 - val_loss: 0.0264
Epoch 7/300
572/572 [==============================] - 1s 2ms/step - loss: 0.0029 - val_loss: 0.0254
Epoch 8/300
572/572 [==============================] - 1s 2ms/step - loss: 0.0026 - val_loss: 0.0237
Epoch 9/300
572/572 [==============================] - 1s 2ms/step - loss: 0.0028 - val_loss: 0.0234
Epoch 10/300
572/572 [========================

Epoch 81/300
572/572 [==============================] - 1s 2ms/step - loss: 0.0016 - val_loss: 0.0110
Epoch 82/300
572/572 [==============================] - 1s 2ms/step - loss: 0.0016 - val_loss: 0.0111
Epoch 83/300
572/572 [==============================] - 1s 2ms/step - loss: 0.0016 - val_loss: 0.0110
Epoch 84/300
572/572 [==============================] - 1s 2ms/step - loss: 0.0016 - val_loss: 0.0112
Epoch 85/300
572/572 [==============================] - 1s 2ms/step - loss: 0.0016 - val_loss: 0.0110
Epoch 86/300
572/572 [==============================] - 1s 2ms/step - loss: 0.0016 - val_loss: 0.0112
Epoch 87/300
572/572 [==============================] - 1s 2ms/step - loss: 0.0016 - val_loss: 0.0110
Epoch 88/300
572/572 [==============================] - 1s 2ms/step - loss: 0.0016 - val_loss: 0.0111
Epoch 89/300
572/572 [==============================] - 1s 2ms/step - loss: 0.0015 - val_loss: 0.0110
Epoch 90/300
572/572 [==============================] - 1s 2ms/step - loss: 0.0015

In [106]:
plot_training_error(history)

### 2.2.3 Evaluate

In [107]:
pred_lstm = model.predict(X_test)
pred_lstm = scaler.inverse_transform(pred_lstm)
y_lstm = scaler.inverse_transform(Y_test)

In [108]:
# RMSE_LSTM = RMSE(flatten(pred_lstm), flatten(y_lstm))
RMSE_LSTM = RMSE(pred_lstm, y_lstm)
print('LSTM RMSE: %.3f' % RMSE_LSTM)

LSTM RMSE: 429.070


In [109]:
X_test_rescaled = list(map(scaler.inverse_transform, X_test))
plot_windows(X_test_rescaled, y_lstm, pred_lstm, WINDOWS_TO_PLOT)

## 2.3 LSTM (chunked)

In [ ]:
def window_slicing(a, L, S ):  # Window len = L, Stride len/stepsize = S
    nrows = ((a.size-L)//S)+1
    return a[S*np.arange(nrows)[:,None] + np.arange(L)]

In [ ]:
array = np.array([1, 2, 3, 4, 5, 6])
print(window_slicing(array, 2, 2))
print(window_slicing(array, 4, 4))

In [ ]:
def train_test_split(array, split):
    train_size = int(array.shape[0] * split)
    test_size = len(array) - train_size
    train, test = array[0:train_size], array[train_size:len(array)]
    return train, test

In [ ]:
array = np.array([1, 2, 3, 4, 5, 6, 7, 8])
example_train, example_test = train_test_split(array, 0.75)
print(example_train, example_test)

In [ ]:
train_start = date(2016, 1, 1)
test_end = date(2017, 10, 20)

df_new = Daily_Price[(Daily_Price.index >= train_start) & (Daily_Price.index <= test_end)]
df_new.tail()

# scale datasets
scaler = MinMaxScaler()
daily_price_scaled = scaler.fit_transform(df_new.Weighted_Price.values.reshape(-1, 1))

In [ ]:
days = 82
split = 0.66

fixed_windows = window_slicing(daily_price_scaled, days, days)
# print(fixed_windows.shape)

window_splits = list(map(lambda window: train_test_split(window, split), fixed_windows))
train_w = [i[0] for i in window_splits]
test_w = [i[1] for i in window_splits]
# print(len(train_w), len(test_w))

In [ ]:
from itertools import chain

def windows_create_lookback(windows, look_back, look_ahead):
    lookbacks = list(map(lambda window: create_lookback(window, look_back, look_ahead), windows))
    x = [i[0] for i in lookbacks]
    y = [i[1] for i in lookbacks]
    x = np.array(list(chain.from_iterable(x)))
    y = np.array(list(chain.from_iterable(y)))
    x = np.reshape(x, (len(x), 1, x.shape[1]))
    y = y[:,:,0]
    return x, y

look_back = 21
look_ahead = 7
    
x_train, y_train = windows_create_lookback(train_w, look_back, look_ahead)
x_test, y_test = windows_create_lookback(test_w, look_back, look_ahead)

# print(x_train.shape, len(x_train),len(x_train[0]), y_train.shape, len(y_train), len(y_train[0]))
# print(x_test.shape, len(x_test), len(x_test[0]), y_test.shape, len(y_test), len(y_test[0]))

dates_fixed_windows = window_slicing(df_new.index.values.reshape(-1, 1), days, days)
# print(dates_fixed_windows.shape)
dates_window_splits = list(map(lambda window: train_test_split(window, split), dates_fixed_windows))
dates_train_w = [i[0] for i in dates_window_splits]
dates_test_w = [i[1] for i in dates_window_splits]
# print(len(dates_train_w), len(dates_test_w))

_, dates_y_train = windows_create_lookback(dates_train_w, look_back, look_ahead)
_, dates_y_test = windows_create_lookback(dates_test_w, look_back, look_ahead)
# print(dates_y_train.shape, len(dates_y_train), dates_y_test.shape, len(dates_y_test))

In [ ]:
#this just assigns the original variables the output from the new approach 
X_train = x_train
Y_train = y_train

X_test = x_test
Y_test = y_test
print(X_train.shape, Y_train.shape, X_test.shape, Y_test.shape)

In [ ]:
model = keras.models.Sequential()
model.add(keras.layers.LSTM(256, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(keras.layers.LSTM(256))
model.add(keras.layers.Dense(look_ahead))

opt = keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=1e-3, amsgrad=False)

model.compile(loss='mean_squared_error', optimizer=opt)
history = model.fit(X_train, Y_train, epochs=300, batch_size=16, shuffle=False,
                    validation_data=(X_test, Y_test),
                    callbacks = [keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=5e-5, patience=20, verbose=1)])

In [ ]:
plot_training_error(history)

In [ ]:
pred_lstm = model.predict(X_test)
pred_lstm = scaler.inverse_transform(pred_lstm)
y_lstm = scaler.inverse_transform(Y_test)

In [ ]:
# RMSE_LSTM = RMSE(flatten(pred_lstm), flatten(y_lstm))
RMSE_LSTM_CHUNKED = RMSE(pred_lstm, y_lstm)
print('LSTM RMSE (chunked): %.3f' % RMSE_LSTM_CHUNKED)

In [ ]:
def plot_windows(X_test, Y_test, predicted, windows_to_plot):
    plot_data = []
    min_y = float('inf')
    max_y = float('-inf')

    for i in range(len(windows_to_plot)):
        w = windows_to_plot[i]
        X = flatten(X_test[w])
        Y = Y_test[w]
        pred = predicted[w]

        look_back = len(X)
        look_ahead = len(Y)
        full_window = look_back + look_ahead

        # first half of window: look back
        plot_data.append(go.Scatter(
            x = np.arange(i * full_window, i * full_window + look_back, 1),
            y = X,
            mode = 'lines+markers',
            name = 'window',
            line = dict(color=('rgb(0, 0, 0)'), width=2)
        ))

        # second half of window: true labels
        plot_data.append(go.Scatter(
            x = np.arange(i * full_window + look_back - 1, i * full_window + full_window, 1),
            y = np.insert(Y, 0, X[-1], axis=0),
            mode = 'lines+markers',
            name = 'True labels',
            line = dict(color=('rgb(66, 244, 155)'), width=2)
        ))

        # second half of window: predicted labels 
        plot_data.append(go.Scatter(
            x = np.arange(i * full_window + look_back - 1, i * full_window + full_window, 1),
            y = np.insert(pred, 0, X[-1], axis=0),
            mode = 'lines+markers',
            name = 'Predicted labels',
            line = dict(color=('rgb(244, 146, 65)'), width=2)
        ))

        local_min = min(min(X), min(Y), min(pred))
        local_max = max(max(X), max(Y), max(pred))

        if (local_min < min_y): min_y = local_min
        if (local_max > max_y): max_y = local_max

    # add vertical lines
    shapes = list()
    for i in range(0, len(windows_to_plot), 1):
        shapes.append({
            'type': 'line',
            'x0': i*28,
            'y0': min_y,
            'x1': i*28,
            'y1': max_y,
            'line': {
                'color': 'rgb(55, 128, 191)',
                'width': 1,
        }})


    layout = dict(shapes=shapes, title = 'Comparison of true prices (on the test dataset) with prices our model predicted',
                 xaxis = dict(title = 'Day number'), yaxis = dict(title = 'Price, USD'))

    fig = dict(data=plot_data, layout=layout)
    py.iplot(fig, filename='results_demonstrating0')

In [ ]:
X_test_rescaled = list(map(scaler.inverse_transform, X_test))

num_windows = 3
windows_to_plot = random.sample(range(len(pred_lstm)), num_windows)
plot_windows(X_test_rescaled, y_lstm, pred_lstm, windows_to_plot)

## 2.4 RMSE comparison

In [ ]:
rmse_x = ['SVM (RBF)', 'Linear regression', 'ARIMA', 'LSTM', 'LSTM (chunked)']
rmse_y = list(map(lambda RMSE: round(RMSE, 2), [RMSE_NAIVE_SVM, RMSE_NAIVE_LINEAR, RMSE_NAIVE_ARIMA, RMSE_LSTM, RMSE_LSTM_CHUNKED]))
# y = list(map(lambda RMSE: round(RMSE, 2), [624.71, 543.97, 463.77, 401.42, 392.146]))

data = []

data.append(go.Bar(
    x = rmse_x,
    y = rmse_y,
    text = rmse_y,
    textposition = 'auto',
    textfont=dict(size=18),
    marker=dict(color=['rgba(204,204,204,0.7)', 'rgba(204,204,204,0.7)', 'rgba(45,100,100,0.7)', 'rgba(45,170,38,0.7)', 'rgba(45,170,38,0.7)']),
    width = [0.7, 0.7, 0.7, 0.7, 0.7, 0.7]
))

layout = dict(
    xaxis = dict(tickfont=dict(size=22)),
    yaxis = dict(title = 'RMSE', titlefont=dict(size=26), tickfont=dict(size=18)))

fig = dict(data=data, layout=layout)
py.iplot(fig, filename='rmse comparison')